# 0. Чтение данных и базовое ознакомление

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

In [2]:
# считываем данные
df = pd.read_csv('data/data.csv')

In [3]:
# выясним размерность датасета
print('размерность датасета: ',df.shape)
# проверим корректность загрузки и ознакомимся с полями
df.head()

размерность датасета:  (377185, 18)


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [4]:
# оценим количество пропусков и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [5]:
# посмотрим на количество явных пропусков с более удобного ракурса
df.isnull().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

Описание данных:

- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

_____________

# 1. Предобработка данных

In [6]:
# для начала удалим записи с пустыми значениями целевой переменной, они явно не участвуют в этом мероприятии
df = df[~df['target'].isna()]
df.shape[0]

374704

In [7]:
# пройдемся по датасету и уберем общую проблему - лишние пробелы и переносы строк
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

In [8]:
# проанализируем датасет на наличие полных дубликатов
df.duplicated().sum()

49

In [9]:
# удалим полные дубликаты исходного датасета
df = df.drop_duplicates()
df.shape[0]

374655

#### Последовательно проверим признаки

In [10]:
# ознакомимся с внесенными значениями по диагонали, чтобы выделить основные сложности с обработкой
columns = df.columns
for column in columns:
    print()
    print(column)
    print('количество вариантов : ', df[column].value_counts().shape[0])
    print()
    print(df[column].value_counts().head(50))
    if df[column].value_counts().shape[0] > 50:
        print(df[column].value_counts().tail(50))


status
количество вариантов :  156

for sale                             156054
Active                               105206
For sale                              43464
foreclosure                            5677
New construction                       5458
Pending                                4697
Pre-foreclosure                        2000
P                                      1488
Pre-foreclosure / auction              1281
Under Contract Show                    1183
/ auction                               799
Under Contract   Showing                793
Active Under Contract                   718
New                                     690
Under Contract                          690
Contingent                              581
Price Change                            563
Auction                                 493
A Active                                443
for rent                                398
Foreclosure                             343
Foreclosed                             

##### Первичный осмотр признаков

- status

156 вариантов - возможно просмотреть и обработать детально. 
Какие-то категории точно можно объединять, вроде “Coming soon”
При этом, объект может менять свои статусы, и какие-то могут влиять на цену (например, как предположение, за долги), а какие-то нет (когда проходит обычный жизненный цикл вроде такого: “для продажи - показы - договор без обязательств”)

- propertyType

Один из самых сложных признаков в выборке. 
Разное написание, разные сокращения, и перечисление признаков через запятые и слэш, а где-то и просто через пробел. Очень неудобно.
Даже при разложении каждой записи этого поля на отдельные составляющие вложенного списка пока конечное количество вариантов непонятно.

- street

нет цели проверить каждый адрес, но топ повторяющихся вариантов можно и нужно просмотреть как минимум на предмет разных написаний “адреса нет”
Навскидку по крайней мере похоже, что в поле и правда вносят адрес.

- baths

Вариантов 226. Многовато, но можно обработать массовые нюансы - убрать типовые наборы слов в начале или конце записи, после чего повторно посмотреть схлопнувшиеся варианты и доработать еще раз.
Уже видны записи с 76 и 241 ванной. Пока тяжело осознать, что это может быть, если  не выброс. Также присутствуют записи через слэш. Надо смотреть подробнее.

- homefacts

Пока только понятно, что надо разложить на отдельные поля этот вложенный список. После этого уже отдельно смотреть что оттуда вытянется.

- fireplace

Разное написание, и где-то цифры, где-то слова. 1653 варианта - много для полноценной ручной обработки с индивидуальным просмотром.
При этом, наблюдаются характерные записи с указанием, газ это, электрика или на дровах/брикетах.
Нужно уменьшать количество вариантов.

- city

2019 нас.пункта. В первом приближении похоже, что внесены города.
Есть вероятность разного написания, но это можно выяснить только при более детальном просмотре.

- schools

списки разной длины, с разным количеством школ и соответствующих им характеристик. Здесь вариант только приводить к какой-то средней оценке. 

- sqft

Площадь объекта - по ощущениям основной показатель, его следует обработать максимально аккуратно.
Видно, что присутствует различное написание единиц измерения, а где-то ЕИ совсем отсутствует. Где-то есть разделитель разрядов, где-то его нет. Есть отдельные записи и с текстовым описанием. 
И конечно, критичны записи с отсутствием информации. Учитывая неаккуратность датасета, эта информация может присутствовать в других полях, необходимо посмотреть подробнее.

- zipcode 

Порадовало, что в верхней части рейтинга популярности значений отсутствует “нет кода” или нулевое значение. Этот факт дает надежду, что сможем определять нахождение объекта более подробно, нежели штат или город. 
Но есть некие значения через дефис. Первая составная часть похожа на основную массу индексов, со второй (после дефиса) пока непонятно.

- beds

1147 вариантов. Разное написание/обозначение слова “кровать”. Но что удивительно, в этом поле кое-где внесены значения площади. Возможно, в форме для заполнения какие-то поля были рядом и неочевидно подписаны, за счет чего сюда вносилась информация о площади (то ли жилой, то ли участка). Надо смотреть.

- state

Выглядит аккуратно, но смущают единичные значения - неужели один объект во всем штате? Надо проверить.

- stories

Знаки плюсов, где-то значения с точками, где-то без точек, где-то словами вроде “One” и т.п. Чистить и смотреть результат.

- mls-id и MlsId

Много повторяющихся значений, хотя по описанию признака казалось, что это уникальный идентификатор. Похоже, что нет, надо посмотреть поближе.

- target

Собственно целевой признак. Есть со значком $, есть без него. Есть плюсы, которые даже если что-то обозначали, мы проигнорируем и удалим.
И бросается в глаза в топ-30 значение $1000 в количестве почти тысяча записей. Тоже надо посмотреть.


Приступим к обработке от простого к сложному

____________

##### Частный бассейн - 'private pool' и 'PrivatePool'

In [11]:
# проверим пару признаков наличия частного бассейна
# для начала проверим варианты внесения информации
display(df['private pool'].value_counts())
display(df['PrivatePool'].value_counts())

Yes    4151
Name: private pool, dtype: int64

yes    28686
Yes    11434
Name: PrivatePool, dtype: int64

In [12]:
# предположим, что это признаки, созданные в базе данных в разное время
# т.е., информация о наличии бассейна есть либо в одном, либо в другом
# в таком случае, не должно быть записей, в которых одновременно указано наличие в обоих полях
# проверим простым способом - переведем наличие в "1" и сложим оба столбца
# если запись и там, и там, получим в этих записях двойки

# для начала переведем записи в единицы и проверим сохранение количеств значений
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['PrivatePool'].value_counts())
df['private pool'] = df['private pool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['private pool'].value_counts())

0    334535
1     40120
Name: PrivatePool, dtype: int64

0    370504
1      4151
Name: private pool, dtype: int64

In [13]:
# поскольку перекодирование столбцов прошло успешно, теперь делаем аггрегирующее поле
# проверка вариантов значений покажет, подтвердилось ли вышеописанное предположение
df['PoolPrivate'] = df['PrivatePool'] + df['private pool']
df['PoolPrivate'].value_counts()

0    330384
1     44271
Name: PoolPrivate, dtype: int64

In [14]:
# схема сработала, изначальные признаки можно удалить
df = df.drop(columns=['private pool', 'PrivatePool'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  mls-id        24937 non-null   object
 14  MlsId         310187 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   374655 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 51.5+ MB


__________

##### Идентификаторы MLS - 'mls-id' и 'MlsId'

In [15]:
# проверим пару признаков идентификаторов MLS
# предположение такое - признаки заполнялись в разные периоды времени (как бассейны)
# если это так, значение есть либо в одном поле, либо в другом, проверим
print('Количество записей с одновременно заполненными полями "MlsId" и "mls-id" : ',df[~df['MlsId'].isna() & ~df['mls-id'].isna()].shape[0])

Количество записей с одновременно заполненными полями "MlsId" и "mls-id" :  0


In [16]:
# нет ни одной записи, в которой одновременно были бы непустые значения в обоих полях
# теперь проверим, нет ли одинаковых идентификаторов в обоих полях, и если есть, то идентичные ли записи им соответствуют
# это будет обозначать, что в какой-то момент одно поле для внесения отключили, а второе активировали
# для начала переведем в строчные буквы
df['MlsId'] = df['MlsId'].str.lower()
df['mls-id'] = df['mls-id'].str.lower()
MlsIdList = list(df['MlsId'].unique())
print('уникальных значений MlsId : ',len(MlsIdList))
mls_id_List = list(df['mls-id'].unique())
print('уникальных значений mls-id : ',len(mls_id_List))
print ('при пересечении только nan длина множества должна быть : ', (len(MlsIdList)+len(mls_id_List)-1))
Mls = set(MlsIdList + mls_id_List)
print('уникальных значений кумулятивно по обоим столбцам : ',len(Mls))
if len(Mls) < (len(MlsIdList)+len(mls_id_List)-1):
    print('значения столбцов пересекаются')
else:
    print('значения столбцов не пересекаются')

уникальных значений MlsId :  232861
уникальных значений mls-id :  24902
при пересечении только nan длина множества должна быть :  257762
уникальных значений кумулятивно по обоим столбцам :  248925
значения столбцов пересекаются


In [17]:
# выделим несколько ID MLS, которые присутствуют в обоих полях, и проверим, одинаковые ли объекты им соответствуют
q = list(set(MlsIdList) & set(mls_id_List))[1:]
df[df['mls-id'].isin(q[:10]) | df['MlsId'].isin(q[:10])].sort_values(by='street')

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate
358682,for sale,single-family home,1 Radney Cir,6 Baths,"{'atAGlanceFacts': [{'factValue': '1968', 'fac...",yes,Houston,"[{'rating': ['8/10', '6/10', '10/10'], 'data':...","6,910 sqft",77024,5 Beds,TX,2.0,NaN,14554484,"$2,450,000",0
56808,NaN,Single Family,1 Radney Cir,6,"{'atAGlanceFacts': [{'factValue': '1968', 'fac...",NaN,Piney Point Village,"[{'rating': ['10/10', '6/10', '8/10'], 'data':...","6,910",77024,5,TX,2.0,NaN,14554484,"$2,450,000",0
218827,For sale,Single Family,1 Radney Cir,6 ba,"{'atAGlanceFacts': [{'factValue': '1968', 'fac...",NaN,Houston,"[{'rating': ['10/10', '6/10', '8/10'], 'data':...","6,910 sqft",77024,5 bd,TX,NaN,14554484,NaN,"$2,450,000",0
146453,for sale,single-family home,1 Radney Cir,6 Baths,"{'atAGlanceFacts': [{'factValue': '1968', 'fac...",yes,Piney Pt Village,"[{'rating': ['8/10', '6/10', '10/10'], 'data':...","6,910 sqft",77024,5 Beds,TX,2.0,NaN,14554484,"$2,450,000",0
312041,for sale,single-family home,1520 NE 190th St,3 Baths,"{'atAGlanceFacts': [{'factValue': '1959', 'fac...",yes,Shoreline,"[{'rating': ['9/10', '6/10', '9/10'], 'data': ...","2,580 sqft",98155,6 Beds,WA,1.0,NaN,1538056,"$645,500",0
172087,For sale,Single Family,1520 NE 190th St,3 ba,"{'atAGlanceFacts': [{'factValue': '1959', 'fac...",NaN,Shoreline,"[{'rating': ['9/10', '6/10', '9/10'], 'data': ...","2,580 sqft",98155,6 bd,WA,NaN,1538056,NaN,"$645,500",0
252719,For sale,Single Family,2976 Plantation Rd,Bathrooms: 3,"{'atAGlanceFacts': [{'factValue': '1988', 'fac...",Yes,Winter Haven,"[{'rating': ['4/10', '3/10', '3/10'], 'data': ...","Total interior livable area: 3,941 sqft",33884,4 bd,FL,1.0,p4908667,NaN,"$524,900",1
130842,for sale,single-family home,2976 Plantation Rd,3 Baths,"{'atAGlanceFacts': [{'factValue': '1988', 'fac...",yes,Winter Haven,"[{'rating': ['3/10', '4/10', '3/10'], 'data': ...","3,941 sqft",33884,4 Beds,FL,1.0,NaN,p4908667,"$524,900",1
256560,for sale,single-family home,315 Nottingham Rd,5 Baths,"{'atAGlanceFacts': [{'factValue': '1981', 'fac...",yes,Jacksonville,"[{'rating': ['5/10', '6/10', '4/10'], 'data': ...","4,960 sqft",28546,6 Beds,NC,2.0,NaN,100181501,"$465,000",0
179939,For sale,Single Family,315 Nottingham Rd,Bathrooms: 5,"{'atAGlanceFacts': [{'factValue': '1981', 'fac...",Yes,Jacksonville,"[{'rating': ['4/10', '6/10', '5/10'], 'data': ...","Total interior livable area: 4,960 sqft",28546,6 bd,NC,2.0,100181501,NaN,"$465,000",0


In [18]:
# на примере выборочной проверки делаем вывод, что идентификаторы в обоих полях относятся к одним и тем же объектам
# объединим идентификатор MLS в один столбец и удалим два исходных
df['MLS'] = df['MlsId'].fillna('') + df['mls-id'].fillna('')
#df = df.drop(columns=['MlsId', 'mls-id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  mls-id        24937 non-null   object
 14  MlsId         310187 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   374655 non-null  int64 
 17  MLS           374655 non-null  object
dtypes: int64(1), object(17)


In [19]:
# посмотрим неоднократно встречающиеся варианты для проверки разного написания отсутствия номера
df['MLS'].value_counts().head(50)

                             39531
no mls                          42
no mls #                        16
a, houston, tx 77008            13
no                              12
12a, orlando, fl 32833          11
b, houston, tx 77008             9
1, south boston, ma 02127        9
b, houston, tx 77007             8
2, washington, dc 20002          8
11a, orlando, fl 32833           8
1, washington, dc 20002          7
2, washington, dc 20010          7
1, washington, dc 20010          6
1, washington, dc 20001          6
a, austin, tx 78721              6
2, washington, dc 20001          6
a, austin, tx 78704              6
2101941                          6
1412350                          6
2088662                          6
3a, orlando, fl 32833            6
1a, orlando, fl 32833            6
2, washington, dc 20009          6
nomlsid                          6
2, boston, ma 02129              6
0, doral, fl 33178               6
a, houston, tx 77018             5
14181176            

In [20]:
# заменим обозначение отсутствия номера MLS на однотипное пустое значение
no_mls = ['no mls', 'no mls #', 'no', 'nomlsid']
df['MLS'] = df['MLS'].apply(lambda x: '' if (x in no_mls) else x)
df['MLS'].value_counts().head(50)

                             39607
a, houston, tx 77008            13
12a, orlando, fl 32833          11
1, south boston, ma 02127        9
b, houston, tx 77008             9
2, washington, dc 20002          8
11a, orlando, fl 32833           8
b, houston, tx 77007             8
2, washington, dc 20010          7
1, washington, dc 20002          7
2, boston, ma 02129              6
1, washington, dc 20010          6
2, washington, dc 20001          6
0, doral, fl 33178               6
a, austin, tx 78704              6
1412350                          6
1, washington, dc 20001          6
3a, orlando, fl 32833            6
a, austin, tx 78721              6
2, washington, dc 20009          6
2088662                          6
1a, orlando, fl 32833            6
2101941                          6
b, houston, tx 77057             5
1026004                          5
2281272                          5
1019437                          5
14168541                         5
1367153             

____________

##### Целевой признак target

In [21]:
# посмотрим еще раз в первом приближении
df['target'].value_counts()

$225,000     1462
$275,000     1355
$250,000     1312
$350,000     1296
$299,900     1276
             ... 
274,359         1
$273,490+       1
$645,000+       1
$28,272         1
$171,306        1
Name: target, Length: 43939, dtype: int64

In [22]:
# проверим, нет ли помимо явных символов еще и букв в каких-то значениях
df[df['target'].str.contains('[a-zA-Z:]')]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate,MLS
547,for rent,single-family home,4323 N Central Park Ave,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",yes,Chicago,"[{'rating': ['1/10', '4/10', '2/10', 'None/10'...","3,300 sqft",60618,4 Beds,IL,NaN,NaN,10588057,"$5,500/mo",0,10588057
609,for rent,multi-family,220 Boylston St #1412,2 Baths,"{'atAGlanceFacts': [{'factValue': '1985', 'fac...",yes,Boston,"[{'rating': [], 'data': {'Distance': [], 'Grad...","1,673 sqft",2116,2 Beds,MA,NaN,NaN,72580936,"$10,500/mo",0,72580936
2075,for rent,single-family home,2830 NE 56th Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '1965', 'fac...",NaN,Fort Lauderdale,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","2,400 sqft",33308,4 Beds,FL,NaN,NaN,a10521855,"$6,390/mo",1,a10521855
3025,for rent,multi-family,411 Kline Aly,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Clarksville,"[{'rating': ['8/10', '9/10', '7/10'], 'data': ...","1,280 sqft",37040,2 Beds,TN,NaN,NaN,2102821,"$1,200/mo",0,2102821
3645,for rent,multi-family,240 E Illinois St #2011,2 Baths,"{'atAGlanceFacts': [{'factValue': '2003', 'fac...",NaN,Chicago,"[{'rating': ['4/10', '7/10'], 'data': {'Distan...","1,473 sqft",60611,2 Beds,IL,NaN,NaN,10590275,"$3,600/mo",1,10590275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371791,for rent,multi-family,9436 Turrentine Dr,1.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,El Paso,"[{'rating': ['4/10', '8/10', '6/10'], 'data': ...","1,050 sqft",79925,2 Beds,TX,NaN,NaN,820163,$890/mo,0,820163
372459,for rent,townhouse,34 Jonquil Pl,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,The Woodlands,"[{'rating': ['5/10', '8/10', '7/10', '8/10'], ...","2,601 sqft",77375,3 Beds,TX,NaN,NaN,62158637,"$2,500/mo",0,62158637
374288,for rent,single-family home,8864 Devonshire Dr,2 Baths,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",yes,Fort Worth,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","2,000 sqft",76131,4 Beds,TX,NaN,NaN,NaN,"$2,000/mo",0,
375550,for rent,townhouse,2217 W Seybert St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '3/10'], 'data': {'Distan...",720 sqft,19121,2 Beds,PA,NaN,NaN,paph857944,"$1,500/mo",0,paph857944


In [23]:
# обнаружились записи со стоимостью аренды в месяц, посмотрим в целом имеющие отношение к аренде записи
df[df['status'].str.contains('rent', na=False)]['status'].value_counts()

for rent              398
Apartment for rent      7
Condo for rent          7
Name: status, dtype: int64

In [24]:
# поскольку в целом модель должна предказывать стоимость продажи объекта
# плюс поскольку даже при желании на 400 записях достойный прогноз не построишь
# и эти записи составляют 0,1% от общего количества
# удаляем все записи, связанные с арендой
df = df[~df['status'].str.contains('rent', na=False)]
df.shape[0]

374243

In [25]:
# теперь заменим все остальные обнаруженные знаки, которые мешают перевести суммы в числовой формат
df['target'] = df['target'].apply(lambda x: int(x.replace('$','').
                                replace('+','').
                                replace(',','')))
# и собственно сменим тип данных в этом признаке
df['target'] = df['target'].astype(int)

________________

In [26]:
# проверим наличие дублей при исключении из датасета того или иного поля
# columns = df.columns
# for column in columns:
#    print(column, ' - ',df.drop(columns=column, axis=1).duplicated().sum())